<a href="https://colab.research.google.com/github/tlsdmswn01/Python/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%A7%88%EC%9D%B4%EB%8B%9D/%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%A0%84%EC%B2%98%EB%A6%AC_%ED%86%A0%ED%81%B0%ED%99%94%2C%ED%83%9C%EA%B9%85%2C_%ED%86%B5%EC%9D%BC%ED%99%94_%ED%95%A8%EC%88%98%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## konlpy 설치

In [5]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.2 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd

import json
import re

from tqdm.notebook import tqdm

In [7]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

In [8]:
import os
from datetime import datetime

## Download Dataset(github에서 NSMC 데이터셋 다운로드)

In [72]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [23]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [57]:
train.head(15)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


# EDA

다양하게 데이터를 탐색해 봅시다.

- 데이터 수는 어떤지?
- 데이터 라벨의 분포는 어떤지?
- NA 데이터는 없는지?
- 중복된 데이터는 없는지?
- 등등

In [37]:
train=train.dropna() #na 값 처리

In [76]:
okt=Okt()

### 전처리 함수 구현

위에서 여러분이 했던 전처리 과정을 하나의 연속된 하나의 과정으로 만들어 볼까요?

한 문장이 들어가서 클리닝 된 리스트로 return 받는 함수를 작성해봅시다.

In [80]:
import nltk
nltk.download('wordnet') #Lemmatization - 표제어 추출(통일화)
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

# 토큰화하기
def preprocessing(s):
    try:
    #[]제거
        s = re.sub(pattern='\[[^)]*\]', repl='', string=s)

    # 특수문자 제거
        s = re.sub('[^a-zA-Z가-힣\s]', repl='', string=s)
    #원하는 불용어 제거하기
        for j in ['목소리','이다','래서']:
            s = re.sub(j, repl='', string=s)
    except:
        print(s)


    # 토큰화
    s_token=okt.morphs(s,norm=True)


    # 글자 1개만 있으면 제외하기
    word_list = []
    for word in s_token:
        if len(word) !=1:
            word_list.append(wl.lemmatize(word))

    return word_list

# 태깅하기
def tagging(token):
    s_token1=[]
    for token1 in token:
        s_token1 += okt.pos(token1)
    return s_token1


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [82]:
X=train.iloc[0:3,:]
a=X['document'].apply(lambda s: preprocessing(s))
a.apply(lambda s: tagging(s))

0         [(더빙, Noun), (진짜, Noun), (짜증나네요, Adjective)]
1    [(포스터, Noun), (보고, Noun), (초딩, Noun), (영화, Nou...
2    [(무재, Noun), (밓었, Noun), (다그, Noun), (보는것을, Ve...
Name: document, dtype: object

In [69]:
train=train.iloc[50000:,:]
train['document'].apply(lambda s: preprocessing(s))

nan


AssertionError: ignored